# ⭐ 20 — FATO_PROCESSO_REGULATORIO (Gold)

Grão: **1 linha = 1 processo**

- Lê `silver/2018_anonimizado.xlsx` e `silver/2019_anonimizado.xlsx`
- Prepara IDs e chaves de tempo
- Cria métricas e flags para BI
- Exporta em `gold/output/fato_processo_regulatorio.csv`


## 0) Imports

In [1]:
import pandas as pd
import numpy as np


## 1) Paths robustos

In [2]:
from pathlib import Path


# ------------------------------------------------------
# Notebook rodando em /gold
# Arquivos de entrada também em /gold
# Saída em /gold/output
# ------------------------------------------------------

BASE_DIR = Path().resolve()   # pasta atual (gold/)
OUT_DIR = BASE_DIR / "output"
OUT_DIR.mkdir(parents=True, exist_ok=True)

INPUT_FILES = [
    BASE_DIR / "2018_anonimizado.xlsx",
    BASE_DIR / "2019_anonimizado.xlsx",
]

print("📁 BASE_DIR:", BASE_DIR)
print("📥 INPUT_FILES:")
for f in INPUT_FILES:
    print(" -", f, "| existe?", f.exists())

print("📤 OUT_DIR:", OUT_DIR)


📁 BASE_DIR: C:\Users\LeaoN\OneDrive\Documents\GitHub\data_case_analysis\gold
📥 INPUT_FILES:
 - C:\Users\LeaoN\OneDrive\Documents\GitHub\data_case_analysis\gold\2018_anonimizado.xlsx | existe? True
 - C:\Users\LeaoN\OneDrive\Documents\GitHub\data_case_analysis\gold\2019_anonimizado.xlsx | existe? True
📤 OUT_DIR: C:\Users\LeaoN\OneDrive\Documents\GitHub\data_case_analysis\gold\output


## 2) Ler Silver (2018 + 2019)

In [3]:
dfs = []
for f in INPUT_FILES:
    if not f.exists():
        raise FileNotFoundError(f"Arquivo não encontrado: {f}")
    tmp = pd.read_excel(f, dtype=str)
    tmp["fonte_arquivo"] = f.name
    dfs.append(tmp)

df = pd.concat(dfs, ignore_index=True)

print("✅ Linhas/Colunas consolidadas:", df.shape)
df.head()


✅ Linhas/Colunas consolidadas: (732261, 87)


,ULTIMO_PROCESSO,SITUACAO_DO_PROCESSO,IS_SEDE_EAD,NO_DO_PROCESSO,MODALIDADE,ANO_DO_PROTOCOLO,DATA,ORGAO,ATO,CATEGORIA_ATO,...,CINE_AREA_ESPECIFICA,CODIGO_AREA_GERAL_CINE,AREA_GERAL_CINE,CODIGO_AREA_DETALHADA_CINE,AREA_DETALHADA_CINE,CODIGO_AREA_ESPECIFICA_CINE,AREA_ESPECIFICA_CINE,ROTULO_CINE,AVALIACAO_OFICIAL,fonte_arquivo
0,NÃO,Aguardando Pagamento,N,200810426,EAD,2009,2009-02-26 00:00:00,NaN,Credenciamento EAD,Instituição,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018_anonimizado.xlsx
1,NÃO,Aguardando Pagamento,N,200810426,EAD,2009,2009-02-26 00:00:00,NaN,Credenciamento EAD,Instituição,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018_anonimizado.xlsx
2,NÃO,Aguardando Pagamento,N,200810426,EAD,2009,2009-02-26 00:00:00,NaN,Credenciamento EAD,Instituição,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018_anonimizado.xlsx
3,NÃO,Aguardando Pagamento,S,200810426,EAD,2009,2009-02-26 00:00:00,NaN,Credenciamento EAD,Instituição,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018_anonimizado.xlsx
4,NÃO,Arquivado,N,20070028,PRESENCIAL,2008,2008-09-26 00:00:00,SERES/DIREG/CGRERCES,Reconhecimento de Curso,Curso,...,Humanidades (exceto línguas),2,Artes e humanidades,223,Filosofia e ética,22,Humanidades (exceto línguas),Filosofia,Regulação,2018_anonimizado.xlsx


## 3) Funções auxiliares

In [4]:


def norm_missing(s: pd.Series) -> pd.Series:
    x = s.astype(str).str.strip()
    return x.replace({"": np.nan, "nan": np.nan, "NAN": np.nan, "None": np.nan, "NONE": np.nan})

def dedup_most_complete(df_in: pd.DataFrame, key: str) -> pd.DataFrame:
    score = df_in.notna().sum(axis=1)
    return (df_in.assign(_score=score)
              .sort_values([key, "_score"], ascending=[True, False])
              .drop_duplicates(subset=[key], keep="first")
              .drop(columns=["_score"]))

def pick_first_existing(candidates, df_):
    return next((c for c in candidates if c in df_.columns), None)


def to_numeric(s: pd.Series):
    return pd.to_numeric(s, errors='coerce')

def mk_date_key(s: pd.Series):
    d = pd.to_datetime(s, errors='coerce', dayfirst=True).dt.normalize()
    return d.dt.strftime('%Y%m%d').astype('Int64'), d


## 4) Construir FATO

In [5]:
fact = df.copy()

fact["id_processo"] = norm_missing(fact["NO_DO_PROCESSO"]) if "NO_DO_PROCESSO" in fact.columns else pd.Series(range(1, len(fact)+1), dtype="Int64")
fact["id_curso"] = norm_missing(fact["CODIGO_DO_CURSO"]) if "CODIGO_DO_CURSO" in fact.columns else pd.NA

if "IES_ID_FAKE" in fact.columns:
    fact["id_ies"] = norm_missing(fact["IES_ID_FAKE"])
elif "CODIGO_DA_IES" in fact.columns:
    fact["id_ies"] = norm_missing(fact["CODIGO_DA_IES"])
else:
    fact["id_ies"] = pd.NA

uf_proc = norm_missing(fact["UF_PROCESSO"]) if "UF_PROCESSO" in fact.columns else pd.Series([pd.NA]*len(fact))
uf_cad  = norm_missing(fact["UF_CADASTRO"]) if "UF_CADASTRO" in fact.columns else pd.Series([pd.NA]*len(fact))
mun_proc = norm_missing(fact["MUNICIPIO_PROCESSO"]) if "MUNICIPIO_PROCESSO" in fact.columns else pd.Series([pd.NA]*len(fact))
mun_cad  = norm_missing(fact["MUNICIPIO_CADASTRO"]) if "MUNICIPIO_CADASTRO" in fact.columns else pd.Series([pd.NA]*len(fact))

fact["uf"] = uf_proc.fillna(uf_cad).str.upper()
fact["municipio"] = mun_proc.fillna(mun_cad).str.upper().replace({"": np.nan}).fillna("NÃO INFORMADO")

# Modalidade norm
if "MODALIDADE" in fact.columns:
    m = norm_missing(fact["MODALIDADE"]).str.upper()
    fact["modalidade_norm"] = m.replace({"SEMI-PRESENCIAL":"SEMIPRESENCIAL"}).fillna("NÃO INFORMADO")
else:
    fact["modalidade_norm"] = "NÃO INFORMADO"

# Datas -> chaves
fact["dt_protocolo_key"], dt_prot = mk_date_key(fact["DATA"]) if "DATA" in fact.columns else (pd.Series([pd.NA]*len(fact), dtype="Int64"), pd.to_datetime(pd.Series([pd.NA]*len(fact)), errors="coerce"))
fact["dt_ultimo_ato_key"], dt_ult = mk_date_key(fact["DATA_DO_ULTIMO_ATO"]) if "DATA_DO_ULTIMO_ATO" in fact.columns else (pd.Series([pd.NA]*len(fact), dtype="Int64"), pd.to_datetime(pd.Series([pd.NA]*len(fact)), errors="coerce"))
fact["dt_entrada_fase_key"], dt_fase = mk_date_key(fact["DATA_DE_ENTRADA_FASE_ATUAL"]) if "DATA_DE_ENTRADA_FASE_ATUAL" in fact.columns else (pd.Series([pd.NA]*len(fact), dtype="Int64"), pd.to_datetime(pd.Series([pd.NA]*len(fact)), errors="coerce"))

# tempo_tramitacao_dias
if "tempo_tramitacao_dias" in fact.columns:
    fact["tempo_tramitacao_dias"] = to_numeric(fact["tempo_tramitacao_dias"])
else:
    fact["tempo_tramitacao_dias"] = (dt_fase - dt_prot).dt.days if ("DATA" in fact.columns and "DATA_DE_ENTRADA_FASE_ATUAL" in fact.columns) else np.nan
fact.loc[fact["tempo_tramitacao_dias"] < 0, "tempo_tramitacao_dias"] = np.nan

# vagas
if "VAGAS_SOLICITADAS_PROCESSO" in fact.columns:
    fact["VAGAS_SOLICITADAS_PROCESSO"] = to_numeric(fact["VAGAS_SOLICITADAS_PROCESSO"])
if "VAGAS_AUTORIZADAS_CADASTRO" in fact.columns:
    fact["VAGAS_AUTORIZADAS_CADASTRO"] = to_numeric(fact["VAGAS_AUTORIZADAS_CADASTRO"])

if "dif_vagas_processo_cadastro" in fact.columns:
    fact["dif_vagas_processo_cadastro"] = to_numeric(fact["dif_vagas_processo_cadastro"])
else:
    if {"VAGAS_SOLICITADAS_PROCESSO","VAGAS_AUTORIZADAS_CADASTRO"}.issubset(fact.columns):
        fact["dif_vagas_processo_cadastro"] = fact["VAGAS_SOLICITADAS_PROCESSO"].fillna(0) - fact["VAGAS_AUTORIZADAS_CADASTRO"].fillna(0)
    else:
        fact["dif_vagas_processo_cadastro"] = np.nan

if "tem_divergencia_vagas" in fact.columns:
    fact["tem_divergencia_vagas"] = to_numeric(fact["tem_divergencia_vagas"]).fillna(0).astype(int)
else:
    fact["tem_divergencia_vagas"] = fact["dif_vagas_processo_cadastro"].fillna(0).ne(0).astype(int)

# flags
if "IS_SEDE_EAD" in fact.columns:
    v = norm_missing(fact["IS_SEDE_EAD"]).str.upper()
    fact["is_sede_ead_flag"] = v.isin(["SIM","S","TRUE","1","EAD"]).astype(int)
else:
    fact["is_sede_ead_flag"] = 0

if "ENDERECO_DIVERGENTE" in fact.columns:
    v = norm_missing(fact["ENDERECO_DIVERGENTE"]).str.upper()
    fact["endereco_divergente_flag"] = v.isin(["SIM","S","TRUE","1"]).astype(int)
else:
    fact["endereco_divergente_flag"] = 0

# CINE geral
fact["cine_area_geral"] = norm_missing(fact["AREA_GERAL_CINE"]).fillna("Não informado") if "AREA_GERAL_CINE" in fact.columns else "Não informado"

# Seleção final
keep = [
    "id_processo","id_ies","id_curso",
    "uf","municipio",
    "modalidade_norm",
    "ANO_DO_PROTOCOLO" if "ANO_DO_PROTOCOLO" in fact.columns else None,
    "dt_protocolo_key","dt_ultimo_ato_key","dt_entrada_fase_key",
    "tempo_tramitacao_dias",
    "VAGAS_SOLICITADAS_PROCESSO" if "VAGAS_SOLICITADAS_PROCESSO" in fact.columns else None,
    "VAGAS_AUTORIZADAS_CADASTRO" if "VAGAS_AUTORIZADAS_CADASTRO" in fact.columns else None,
    "dif_vagas_processo_cadastro","tem_divergencia_vagas",
    "is_sede_ead_flag","endereco_divergente_flag",
    "cine_area_geral",
    "ATO" if "ATO" in fact.columns else None,
    "CATEGORIA_ATO" if "CATEGORIA_ATO" in fact.columns else None,
    "ORGAO" if "ORGAO" in fact.columns else None,
    "FASE_ATUAL" if "FASE_ATUAL" in fact.columns else None,
    "SITUACAO_DO_PROCESSO" if "SITUACAO_DO_PROCESSO" in fact.columns else None,
]
keep = [c for c in keep if c is not None and c in fact.columns]
fato = fact[keep].copy()

print("✅ FATO pronto:", fato.shape)
fato.head(10)


C:\Users\LeaoN\AppData\Local\Temp\ipykernel_28348\3537942072.py:20: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  d = pd.to_datetime(s, errors='coerce', dayfirst=True).dt.normalize()


✅ FATO pronto: (732261, 23)


,id_processo,id_ies,id_curso,uf,municipio,modalidade_norm,ANO_DO_PROTOCOLO,dt_protocolo_key,dt_ultimo_ato_key,dt_entrada_fase_key,...,dif_vagas_processo_cadastro,tem_divergencia_vagas,is_sede_ead_flag,endereco_divergente_flag,cine_area_geral,ATO,CATEGORIA_ATO,ORGAO,FASE_ATUAL,SITUACAO_DO_PROCESSO
0,200810426,3448,NaN,MG,MONTES CLAROS,EAD,2009,20090226,20070406,<NA>,...,0.0,0,0,0,Não informado,Credenciamento EAD,Instituição,NaN,NaN,Aguardando Pagamento
1,200810426,3448,NaN,MG,BETIM,EAD,2009,20090226,20070406,<NA>,...,0.0,0,0,1,Não informado,Credenciamento EAD,Instituição,NaN,NaN,Aguardando Pagamento
2,200810426,3448,NaN,MG,BELO HORIZONTE,EAD,2009,20090226,20070406,<NA>,...,0.0,0,0,1,Não informado,Credenciamento EAD,Instituição,NaN,NaN,Aguardando Pagamento
3,200810426,3448,NaN,MG,MONTES CLAROS,EAD,2009,20090226,20070406,<NA>,...,0.0,0,1,0,Não informado,Credenciamento EAD,Instituição,NaN,NaN,Aguardando Pagamento
4,20070028,3675,86920,AC,RIO BRANCO,PRESENCIAL,2008,20080926,<NA>,20180920,...,115.0,1,0,0,Artes e humanidades,Reconhecimento de Curso,Curso,SERES/DIREG/CGRERCES,ARQUIVAMENTO NA SECRETARIA,Arquivado
5,20070073,1396,21489,PR,FOZ DO IGUAÇU,PRESENCIAL,2007,20070328,<NA>,20110210,...,-90.0,1,0,0,Educação,Reconhecimento de Curso,Curso,ADMIN,ARQUIVAMENTO PELA IES,Arquivado
6,20070075,10086,NaN,SC,JOINVILLE,PRESENCIAL,2007,20070510,<NA>,20101126,...,0.0,0,0,1,Não informado,Credenciamento,Instituição,SETEC/DEPT/CGRET,SECRETARIA - RECURSO,Arquivado
7,20070098,1396,21489,PR,FOZ DO IGUAÇU,PRESENCIAL,2007,20070328,<NA>,20080325,...,-90.0,1,0,0,Educação,Reconhecimento de Curso,Curso,SESU/DESUP/CGFP (INATIVO),ARQUIVAMENTO PELA IES,Arquivado
8,20070101,1675,NaN,PE,JABOATÃO DOS GUARARAPES,PRESENCIAL,2007,20070530,<NA>,20100311,...,0.0,0,0,0,Não informado,Recredenciamento,Instituição,SESU/DESUP/COREG (INATIVO),SECRETARIA - RECURSO,Arquivado
9,20070101,1675,NaN,PE,JABOATÃO DOS GUARARAPES,PRESENCIAL,2007,20070530,<NA>,20100311,...,0.0,0,0,0,Não informado,Recredenciamento,Instituição,SESU/DESUP/COREG (INATIVO),SECRETARIA - RECURSO,Arquivado


## 5) Exportar

In [6]:
out_file = OUT_DIR / "fato_processo_regulatorio.csv"
fato.to_csv(out_file, index=False, encoding="utf-8")
print("✅ Salvo em:", out_file)


✅ Salvo em: C:\Users\LeaoN\OneDrive\Documents\GitHub\data_case_analysis\gold\output\fato_processo_regulatorio.csv
